# Automatic generation of Notebook using PyCropML
    This notebook implements a crop model.

### Model Updatecalendar

In [ ]:
MODULE list_sub
    IMPLICIT NONE
    TYPE container
        CLASS(*), ALLOCATABLE :: item
        CLASS(*), ALLOCATABLE :: items(:)
    END TYPE

    interface Add
        module procedure AddToListFloat
        module procedure AddToListInt
        module procedure AddToListChar
        module procedure AddToListIntArray
    end interface
CONTAINS

    FUNCTION indice(vectorElem, elem)
        CHARACTER(LEN=*), DIMENSION(:):: vectorElem
        INTEGER::iterator, indice
        CHARACTER(LEN=*):: elem
        DO iterator= 1, SIZE(vectorElem)
            IF(vectorElem(iterator)==elem) THEN
                indice = iterator
            END IF
        END DO
        RETURN
    END FUNCTION indice


    SUBROUTINE AddToListFloat(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize
        REAL, INTENT(IN) :: element
        REAL, DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        REAL, DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            ALLOCATE(list(1))
            list(1) = element
        END IF
    END SUBROUTINE AddToListFloat

    SUBROUTINE AddToListInt(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize
        INTEGER, INTENT(IN) :: element
        INTEGER, DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        INTEGER, DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            ALLOCATE(list(1))
            list(1) = element
        END IF
    END SUBROUTINE AddToListInt

    SUBROUTINE AddToListChar(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize, l
        CHARACTER(LEN=*), INTENT(IN) :: element
        CHARACTER(LEN=*), DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        CHARACTER(LEN=65), DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            l=1
            ALLOCATE(list(l))
            list(l) = element
        END IF
    END SUBROUTINE AddToListChar


    SUBROUTINE AddToListIntArray(a, e)
        INTEGER, ALLOCATABLE, INTENT(INOUT) :: a(:)
        INTEGER, ALLOCATABLE, INTENT(IN) :: e(:)
        INTEGER, ALLOCATABLE:: tmp(:)

        IF (.NOT.ALLOCATED(a)) THEN
            ALLOCATE(a(1))
            ALLOCATE(a(SIZE(e)), source = e)
        ELSE
            CALL MOVE_ALLOC(a,tmp)
            ALLOCATE(a(SIZE(tmp)+SIZE(e)), source = e)
            a(1:SIZE(tmp)) = tmp
            a(SIZE(tmp)+1:SIZE(a)) = e
        END IF
    END SUBROUTINE AddToListIntArray

END MODULE list_sub
MODULE Updatecalendarmod
    USE list_sub
    IMPLICIT NONE
CONTAINS

    SUBROUTINE model_updatecalendar(cumulTT, &
        calendarMoments, &
        calendarDates, &
        calendarCumuls, &
        currentdate, &
        phase)
        IMPLICIT NONE
        REAL, INTENT(IN) :: cumulTT
        CHARACTER(65), ALLOCATABLE , DIMENSION(:), INTENT(INOUT) ::  &
                calendarMoments
        CHARACTER(65), ALLOCATABLE , DIMENSION(:), INTENT(INOUT) ::  &
                calendarDates
        REAL, ALLOCATABLE , DIMENSION(:), INTENT(INOUT) :: calendarCumuls
        CHARACTER(65), INTENT(IN) :: currentdate
        REAL, INTENT(IN) :: phase
        !- Description:
    !            * Title: Calendar Model
    !            * Author: Pierre Martre
    !            * Reference: Modeling development phase in the 
    !                Wheat Simulation Model SiriusQuality.
    !                See documentation at http://www1.clermont.inra.fr/siriusquality/?page_id=427
    !            * Institution: INRA Montpellier
    !            * Abstract: Lists containing for each stage the date it occurs as well as a copy of all types of cumulated thermal times 
        !- inputs:
    !            * name: cumulTT
    !                          ** description : cumul thermal times at current date
    !                          ** variablecategory : auxiliary
    !                          ** datatype : DOUBLE
    !                          ** min : -200
    !                          ** max : 10000
    !                          ** default : 741.510096671757
    !                          ** unit : °C d
    !                          ** inputtype : variable
    !            * name: calendarMoments
    !                          ** description : List containing apparition of each stage
    !                          ** variablecategory : state
    !                          ** datatype : STRINGLIST
    !                          ** default : ['Sowing']
    !                          ** unit : 
    !                          ** inputtype : variable
    !            * name: calendarDates
    !                          ** description : List containing  the dates of the wheat developmental phases
    !                          ** variablecategory : state
    !                          ** datatype : DATELIST
    !                          ** default : ['2007/3/21']
    !                          ** unit : 
    !                          ** inputtype : variable
    !            * name: calendarCumuls
    !                          ** description : list containing for each stage occured its cumulated thermal times
    !                          ** variablecategory : state
    !                          ** datatype : DOUBLELIST
    !                          ** default : [0.0]
    !                          ** unit : °C d
    !                          ** inputtype : variable
    !            * name: currentdate
    !                          ** description : current date
    !                          ** variablecategory : auxiliary
    !                          ** datatype : DATE
    !                          ** default : 2007/3/27
    !                          ** unit : 
    !                          ** inputtype : variable
    !            * name: phase
    !                          ** description :  the name of the phase
    !                          ** variablecategory : state
    !                          ** datatype : DOUBLE
    !                          ** min : 0
    !                          ** max : 7
    !                          ** default : 1
    !                          ** unit : 
    !                          ** inputtype : variable
        !- outputs:
    !            * name: calendarMoments
    !                          ** description :  List containing apparition of each stage
    !                          ** variablecategory : state
    !                          ** datatype : STRINGLIST
    !                          ** unit : 
    !            * name: calendarDates
    !                          ** description :  List containing  the dates of the wheat developmental phases
    !                          ** variablecategory : state
    !                          ** datatype : DATELIST
    !                          ** unit : 
    !            * name: calendarCumuls
    !                          ** description :  list containing for each stage occured its cumulated thermal times
    !                          ** variablecategory : state
    !                          ** datatype : DOUBLELIST
    !                          ** unit : °C d
        IF(phase .GE. 1.0 .AND. phase .LT. 2.0 .AND. ALL(calendarMoments .NE.  &
                'Emergence')) THEN
            call Add(calendarMoments, 'Emergence')
            call Add(calendarCumuls, cumulTT)
            call Add(calendarDates, currentdate)
        ELSE IF ( phase .GE. 2.0 .AND. phase .LT. 3.0 .AND.  &
                ALL(calendarMoments .NE. 'FloralInitiation')) THEN
            call Add(calendarMoments, 'FloralInitiation')
            call Add(calendarCumuls, cumulTT)
            call Add(calendarDates, currentdate)
        ELSE IF ( phase .GE. 3.0 .AND. phase .LT. 4.0 .AND.  &
                ALL(calendarMoments .NE. 'Heading')) THEN
            call Add(calendarMoments, 'Heading')
            call Add(calendarCumuls, cumulTT)
            call Add(calendarDates, currentdate)
        ELSE IF ( phase .EQ. 4.0 .AND. ALL(calendarMoments .NE. 'Anthesis'))  &
                THEN
            call Add(calendarMoments, 'Anthesis')
            call Add(calendarCumuls, cumulTT)
            call Add(calendarDates, currentdate)
        ELSE IF ( phase .EQ. 4.5 .AND. ALL(calendarMoments .NE.  &
                'EndCellDivision')) THEN
            call Add(calendarMoments, 'EndCellDivision')
            call Add(calendarCumuls, cumulTT)
            call Add(calendarDates, currentdate)
        ELSE IF ( phase .GE. 5.0 .AND. phase .LT. 6.0 .AND.  &
                ALL(calendarMoments .NE. 'EndGrainFilling')) THEN
            call Add(calendarMoments, 'EndGrainFilling')
            call Add(calendarCumuls, cumulTT)
            call Add(calendarDates, currentdate)
        ELSE IF ( phase .GE. 6.0 .AND. phase .LT. 7.0 .AND.  &
                ALL(calendarMoments .NE. 'Maturity')) THEN
            call Add(calendarMoments, 'Maturity')
            call Add(calendarCumuls, cumulTT)
            call Add(calendarDates, currentdate)
        END IF
    END SUBROUTINE model_updatecalendar

END MODULE
PROGRAM test
    USE Updatecalendarmod
    REAL:: cumulTT
    CHARACTER(65), ALLOCATABLE, DIMENSION(:):: calendarMoments
    CHARACTER(65), ALLOCATABLE, DIMENSION(:):: calendarDates
    REAL, ALLOCATABLE, DIMENSION(:):: calendarCumuls
    CHARACTER(65):: currentdate
    REAL:: phase
    print *, "--------test_test_wheat1_UpdateCalendar-------"
    cumulTT =  112.330110409888
    CALL Add(calendarMoments, "Sowing")
    CALL Add(calendarDates, "2007/3/21")
    calendarCumuls = [0.0]
    phase = 1
    currentdate = "2007/3/27"
    call model_updatecalendar(cumulTT, calendarMoments, calendarDates,  &
            calendarCumuls, currentdate, phase)
    !calendarMoments: ["Sowing", "Emergence"]
    print *, "calendarMoments estimated :"
    Do i_cyml = 1, size(calendarMoments)
        print *, calendarMoments(i_cyml);
    END DO
    !calendarDates: ["2007/3/21", "2007/3/27"]
    print *, "calendarDates estimated :"
    Do i_cyml = 1, size(calendarDates)
        print *, calendarDates(i_cyml);
    END DO
    !calendarCumuls: [ 0.0 ,112.33]
    print *, "calendarCumuls estimated :"
    Do i_cyml = 1, size(calendarCumuls)
        print *, calendarCumuls(i_cyml);
    END DO

END PROGRAM
